# CSC_52081_EP Project - Team 8

Advanced Machine Learning and Autonomous Agents Project

## Introduction

### Authors and Acknowledgmentss

This project was developed by in alphabetical order:

- Guilherme NUNES TROFINO:
    - [![LinkedIn](https://i.stack.imgur.com/gVE0j.png) LinkedIn](https://www.linkedin.com/in/guilherme-trofino/)
    - [![GitHub](https://i.stack.imgur.com/tskMh.png) GitHub](https://github.com/tr0fin0)

- Mateus GALVÃO:
    - [![LinkedIn](https://i.stack.imgur.com/gVE0j.png) LinkedIn](https://www.linkedin.com/in/galvao-mateus/)
    - [![GitHub](https://i.stack.imgur.com/tskMh.png) GitHub](https://github.com/Mateushga1)

- Rafael BENZI:
    - [![LinkedIn](https://i.stack.imgur.com/gVE0j.png) LinkedIn](https://www.linkedin.com/in/rafael-benzi/)
    - [![GitHub](https://i.stack.imgur.com/tskMh.png) GitHub](https://github.com/rafinhafpb)

We greatly appreciate our [CSC_52081_EP](https://synapses.polytechnique.fr/catalogue/2024-2025/ue/637/CSC-52081-EP-apprentissage-automatique-avance-et-agents-autonomes?from=D1) teachers at [École Polytechnique](https://www.polytechnique.edu/) in alphabetical order:

- Clément ELLIKER;
- Jérémie DECOCK;
- Jesse READ;
- Sonia VANIER;

## Installation

## Setup the Python environment

This notebook relies on several libraries including `cma`, `gymnasium[box2d]` (v1.0.0), `ipywidgets`, `matplotlib`, `moviepy`, `numpy`, `pandas`, `pygame`, `seaborn`, and `tqdm`.
A complete list of dependencies can be found in the following [requirements-lab7.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main/requirements-lab7.txt) file.

To set up the necessary dependencies, select your Python interpreter of preference (can be a [Python virtual environment (venv)](https://docs.python.org/3/library/venv.html)) and run the following code.

### Import required packages

In [5]:
# Verify installation and import packages
import importlib
import subprocess
import sys

def install_and_import(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to check and install if necessary
packages = ["gymnasium", "pandas", "ipywidgets", "matplotlib", "moviepy", "numpy", "pygame", "seaborn", "tqdm", "swig", "Box2D", "itertools", "torch", "cma"]

for package in packages:
    install_and_import(package)

gymnasium is already installed.
pandas is already installed.
ipywidgets is already installed.
matplotlib is already installed.
moviepy is already installed.
numpy is already installed.
pygame is already installed.
seaborn is already installed.
tqdm is already installed.
swig is already installed.
Box2D is already installed.
itertools is already installed.
torch is already installed.
cma is already installed.


In [6]:
import gymnasium as gym
import math
import numpy as np
import pandas as pd
from pathlib import Path
import itertools
import torch
from typing import cast, List, Tuple, Deque, Optional, Callable

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

In [8]:
from IPython.display import Video
from ipywidgets import interact

In [9]:
sns.set_context("talk")

In [10]:
def video_selector(file_path: List[Path]) -> Video:
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

### Test the Environment

In [11]:
FIGS_DIR = Path("figs/videos") / "project"       # Where to save videos (.gif or .mp4 files)
PLOTS_DIR = Path("figs/graphs") / "project"      # Where to save plots (.png or .svg files)
MODELS_DIR = Path("models/") / "project"         # Where to save models (.pth files)

In [12]:
if not FIGS_DIR.exists():
    FIGS_DIR.mkdir(parents=True)
if not PLOTS_DIR.exists():
    PLOTS_DIR.mkdir(parents=True)
if not MODELS_DIR.exists():
    MODELS_DIR.mkdir(parents=True)

In [13]:
VIDEO_PREFIX_TEST = "project_test"

(FIGS_DIR / f"{VIDEO_PREFIX_TEST}-episode-0.mp4").unlink(missing_ok=True)

env = gym.make("CarRacing-v3", continuous=False, render_mode="rgb_array", domain_randomize=False)
env = gym.wrappers.RecordVideo(env, video_folder=str(FIGS_DIR), name_prefix=VIDEO_PREFIX_TEST)

observation, info = env.reset()
done = False

while not done:
    action = env.action_space.sample()   # Random policy

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()

Video(
    FIGS_DIR / f"{VIDEO_PREFIX_TEST}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

c:\ENSTA\Avanced_machine_learning\env-project\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at c:\ENSTA\Avanced_machine_learning\ensta_CSC_52081_EP_project\src\figs\videos\project folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Description